In [47]:
# Join all the strings into one and lowercase it
# Put result into variable text.
import string
import re
import random


# Your great code here
all_characters = string.printable
n_characters = len(all_characters)

with open('onegin.txt', 'r') as iofile:
    text = iofile.readlines()
text = ''.join(text).lower()


text = text.split('\n')

new_text = list()
for sent in text:
    if sent != '' and (sent.endswith("i") or sent.endswith("v") or sent.endswith("x")):
        #new_text[-1] = new_text[-1] + 'eoeos'
         new_text.append('eoeos')
        
    elif sent != '':
        sent = 'sosos' + sent
        new_text.append(sent)

tokens = list()
for i in new_text:
    tokens.extend(re.findall(r"[\w']+|[.,!?;]", i))

tokens.append('\n\n')
tokens = sorted(set(tokens))
tokens

['\n\n',
 '!',
 ',',
 '.',
 '77',
 '78',
 '79',
 ';',
 '?',
 'a',
 'abbe',
 'alia',
 'ange',
 'annette',
 'au',
 'beef',
 'belle',
 'bene',
 'benedetta',
 'ces',
 'cetera',
 'child',
 'comme',
 'crirez',
 'dandy',
 'de',
 'du',
 'e',
 'endormie',
 'entrechat',
 'eoeos',
 'et',
 'far',
 'faut',
 'guillot',
 'harold',
 'idol',
 'il',
 'in',
 'l',
 'madame',
 'mio',
 'mon',
 'monsieur',
 'n',
 'niente',
 'nina',
 'pachette',
 'poor',
 'priest',
 'qu',
 'quarto',
 're',
 'roast',
 'scott',
 'sel',
 'sempre',
 'sosos',
 'sososl',
 'sososxl',
 'sososxxxix',
 'sososглава',
 'sososписьмо',
 'st',
 'staёl',
 'steaks',
 'sur',
 't',
 'tablettes',
 'talon',
 'tatiana',
 'te',
 'tempora',
 'v',
 'vale',
 'veillez',
 'vous',
 'vulgar',
 'walter',
 'xl',
 'yorick',
 'а',
 'авось',
 'авроры',
 'автомедоны',
 'автор',
 'ага',
 'агафон',
 'ада',
 'адама',
 'адель',
 'адриатические',
 'адских',
 'азбуке',
 'азбучным',
 'академиком',
 'академический',
 'акаций',
 'аккорды',
 'аккуратный',
 'актрис',
 'ак

In [48]:
num_tokens = len(tokens)

# dict <index>:<char>
# Your great code here
token_to_idx = {}

for i in range(num_tokens):
    token_to_idx[tokens[i]] = i

assert len(tokens) == len(token_to_idx), "dictionaries must have same size"

# dict <char>:<index>
# Your great code here
idx_to_token = {v: k for k, v in token_to_idx.items()}


# Подача знаков препинания как отдельных токенов
for i in range(len(new_text)):
    new_text[i] = re.findall(r"[\w']+|[.,!?;]", new_text[i])

# Подготовка текста для сплавления в to_matrix по 8 строчек
j = 1
text2 = [new_text[0]]
for i in range(len(new_text)):
    for k in range(len(new_text[i])):
        if new_text[i][k] == 'eoeos':
            new_text[i][k] = '\n\n'
        
    if j == 8:
        text2.append(new_text[i])
        j = 0
    else:
        text2[-1] = text2[-1] + new_text[i]
        j+=1

        
import numpy as np

def to_matrix(names, max_len=None, pad=token_to_idx['eoeos'], dtype='int32', batch_first = True):
    """Casts a list of names into rnn-digestable matrix"""
    
    max_len = max_len or max(map(len, names))
    names_ix = np.zeros([len(names), max_len], dtype) + pad

    for i in range(len(names)):
        line_ix = [token_to_idx[c] for c in names[i]]
        names_ix[i, :len(line_ix)] = line_ix
        
    if not batch_first: # convert [batch, time] into [time, batch]
        names_ix = np.transpose(names_ix)

    return names_ix

In [51]:
# Your code here
import torch, torch.nn as nn
import torch.nn.functional as F
from IPython.display import clear_output
from random import sample

class CharRNNLoop(nn.Module):
    def __init__(self, num_tokens=num_tokens, emb_size=16, rnn_num_units=64):
        super(self.__class__, self).__init__()
        self.emb = nn.Embedding(num_tokens, emb_size)
        self.rnn = nn.RNN(emb_size, rnn_num_units, batch_first=True)
        self.hid_to_logits = nn.Linear(rnn_num_units, num_tokens)
        self.num_units = rnn_num_units

    def forward(self, x):
        h_seq, _ = self.rnn(self.emb(x))
        next_logits = self.hid_to_logits(h_seq)
        next_logp = F.log_softmax(next_logits, dim=-1)
        return next_logp

    def forward_char(self, x, h=None):
        if h is None:
            out, h = self.rnn(self.emb(x))
        else:
            out, h = self.rnn(self.emb(x), h)
        out = self.hid_to_logits(out)
        return out, h

    def initial_state(self, batch_size):
        """ return rnn state before it processes first input (aka h0) """
        return torch.zeros(batch_size, self.num_units, requires_grad=True)

MAX_LENGTH = 16
    
model = CharRNNLoop()
opt = torch.optim.Adam(model.parameters())
history = []

# the model applies over the whole sequence
batch_ix = to_matrix(text2)
batch_ix = torch.LongTensor(batch_ix)

logp_seq = model(batch_ix)

loss = F.nll_loss(
    logp_seq[:, :-1].contiguous().view(-1, num_tokens), batch_ix[:, 1:].contiguous().view(-1)
)

loss.backward()

In [6]:
def generate_text(char_rnn, tokens, seed_phrase="sosos", max_length=MAX_LENGTH, temperature=1.0, device=None):
    '''
    The function generates text given a phrase of length at least SEQ_LENGTH.
    :param seed_phrase: prefix characters. The RNN is asked to continue the phrase
    :param max_length: maximum output length, including seed_phrase
    :param temperature: coefficient for sampling.  higher temperature produces more chaotic outputs,
                        smaller temperature converges to the single most likely output
    '''
    seed_phrase = ''.join([x for x in seed_phrase if x not in string.punctuation])
    seed_phrase = seed_phrase.split(' ')
    x_sequence = [token_to_idx[token] if token in tokens else print('Я не знаю слово "' + token +'"') for token in seed_phrase]
    x_sequence = [x for x in x_sequence if x is not None]
    print(x_sequence)
    x_sequence = torch.tensor([x_sequence], dtype=torch.int64).to(device)
    
    #feed the seed phrase, if any
    logits, h = char_rnn.forward_char(x_sequence)
    logits = logits[:, -1]
    
    #start generating
    for _ in range(max_length - x_sequence.shape[1] + 1):
        p_next = F.softmax(logits / temperature, dim=-1).cpu().data.numpy().squeeze()
        
        # sample next token and push it back into x_sequence
        next_ix = np.random.choice(num_tokens,p=p_next)
        next_ix = torch.tensor([[next_ix]], dtype=torch.int64).to(device)
        x_sequence = torch.cat([x_sequence, next_ix], dim=1)
        logits, h = char_rnn.forward_char(next_ix, h)
    
    from itertools import groupby
    
    return ' '.join([x for x, _ in groupby(['' if y == 'eoeos' else y for y in ['\n' if x == 'sosos' else x for x in [tokens[ix] for ix in x_sequence.cpu().data.numpy()[0]]]])]), seed_phrase

In [7]:
# model_up = CharRNNLoop()
# model_up.load_state_dict(torch.load('text_generation_pushkin_RNN.h5'))
# model_up.eval()


 когда мне порукой моей грустить , 
 и дам с мильонной безотрадной 
 хладные неправедно гонимый день ,  герои 
 здесь мой пасмурный в посвятил 
 не минуте друг найти спят и поэта , 
 как он задремал ; 
 боже ли . у домовых 
 старушка сомненья 
 а на прикажешь он , 
 вас неземной уж путь . 
 пред этой мерзлой медведь сном . 
 неужто бедная на успех ей 
 бледна кабинет , и умом 
 иль благу браздах , любя сон ; 
 и день точь не любя ? . прекрасный 
 не бокалов ленский моей ; 
 татьяна


In [9]:
# print(generate_text(model_up, seed_phrase = 'sosos когда', max_length=150, temperature=1.0))


 когда модных поход очи нас . 
 иль утомить им лесной . 
 бледна что и в летний опустелом , 
 что муж по нравились . 
 под скотт же на свет 
 чистосердечней , вперед , изнывать , 
 любви стали переплывал , не муку 
